# imports

In [96]:
import os, json
import openai
from dotenv import dotenv_values, load_dotenv
from openai.types.chat.chat_completion import ChatCompletion
from IPython.display import display, Markdown


# load api key

In [97]:
load_dotenv()
client = openai.OpenAI()

# functions
## openai helper function

In [98]:
def get_llm_response(prompt:str, model:str='gpt-4o-mini', return_dict_body:bool=False, return_response_body:bool=False) -> str | dict | ChatCompletion:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    
    # print(type(response))
    
    if return_response_body:
        return response
    if return_dict_body:
        return response.model_dump()
    return response.choices[0].message.content


## prompt template interpolate function

In [99]:
def get_colors_prompt(prompt:str) -> str:
    return f'''
    You are a color palette generating assistant that responds to text prompts for color palettes
    Your should generate color palettes that fit the theme, mood, or instructions in the prompt.
    The palettes should be between 2 and 8 colors, the more the better.

    Q: Convert the following verbal description of a color palette into a list of colors: The Mediterranean Sea
    A: ["#006699", "#66CCCC", "#F0E68C", "#008000", "#F08080"]

    Q: Convert the following verbal description of a color palette into a list of colors: sage, nature, earth
    A: ["#EDF1D6", "#9DC08B", "#609966", "#40513B"]


    Desired Format: just a JSON array of hexadecimal color codes, nothing else before or after

    Q: Convert the following verbal description of a color palette into a list of colors: {prompt}
    A:    
    '''


## notebook markdown display helper

In [100]:
def display_colors(colors, width:int=4):
    display(
        Markdown(
            ' '.join(
                f'<span style="color: {color}">{chr(9608) * width}</span>'
                for color in colors
            )
        )
    )

# main
## get json color array

In [101]:
response = get_llm_response(
    get_colors_prompt('blue ocean')
)
response

'["#0077BE", "#00BFFF", "#1E90FF", "#5F9EA0", "#ADD8E6"]'

## convert json to python array

In [102]:
colors = json.loads(response)
colors

['#0077BE', '#00BFFF', '#1E90FF', '#5F9EA0', '#ADD8E6']

## display result

In [103]:
display_colors(colors)

<span style="color: #0077BE">████</span> <span style="color: #00BFFF">████</span> <span style="color: #1E90FF">████</span> <span style="color: #5F9EA0">████</span> <span style="color: #ADD8E6">████</span>

# macro helper function

In [104]:
from json import JSONDecodeError


def show_palette(prompt:str):
    try:
        userprompt = get_colors_prompt(prompt)
        response = get_llm_response(userprompt)
        colors = json.loads(response)
        display_colors(colors)
        print(colors)
    except JSONDecodeError as jde:
        print(f'something wrong with llm response format: {response!r}')

In [105]:
show_palette('ocean abyss')

<span style="color: #003D5B">████</span> <span style="color: #006B83">████</span> <span style="color: #0B8B9B">████</span> <span style="color: #12B0A8">████</span> <span style="color: #D9BF77">████</span> <span style="color: #F9AFAF">████</span>

['#003D5B', '#006B83', '#0B8B9B', '#12B0A8', '#D9BF77', '#F9AFAF']


In [106]:
show_palette('cyber terminal with green chars')

<span style="color: #00FF00">████</span> <span style="color: #1F1F1F">████</span> <span style="color: #333333">████</span> <span style="color: #66FF66">████</span> <span style="color: #AFAFAF">████</span>

['#00FF00', '#1F1F1F', '#333333', '#66FF66', '#AFAFAF']


In [ ]:
show_palette('spongebob')

<span style="color: #F6EB61">████</span> <span style="color: #F4C300">████</span> <span style="color: #BAE4EA">████</span> <span style="color: #D69A20">████</span> <span style="color: #C41400">████</span>

['#F6EB61', '#F4C300', '#BAE4EA', '#D69A20', '#C41400']


In [108]:
show_palette('superman')

<span style="color: #C8102E">████</span> <span style="color: #005EB8">████</span> <span style="color: #F7EA00">████</span> <span style="color: #FFFFFF">████</span> <span style="color: #000000">████</span>

['#C8102E', '#005EB8', '#F7EA00', '#FFFFFF', '#000000']
